This is my Project Notebook

In [8]:
import pandas as pd
import numpy as np
from geopy import geocoders
from geopy.geocoders import GoogleV3
import folium
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print("This is for my capstone project")



This is for my capstone project


In [207]:
df = pd.read_csv('Schools.csv')

sf_dict = {'Name': df['Campus Name'], 'Address': df['Campus Address'], 'Location': df['Location 1'], 'County Name': df['County Name'], 'Grade Range': df['Grade Range'], 'District': df['Supervisor District']}

sf_data = pd.DataFrame(data = sf_dict)

sf_data = sf_data[sf_data['Grade Range'] == '9-12'].reset_index(drop=True)

geolocator = Nominatim(user_agent="sf_explorer")

sf_data['Location'] = sf_data['Location'].map(lambda x: x.lstrip('POINT (').rstrip('aAbBcC'))

lat = []
lon = []

for loc in sf_data['Location']:
    lat.append(float(str.split(loc)[0]))

for loc in sf_data['Location']:
    lon.append(float((str.split(loc)[1])[0:-1]))  
    
sf_data['Latitude'] = lon

sf_data['Longitude'] = lat

sf_data = sf_data.drop(columns = 'Location')

sf_data

,Name,Address,County Name,Grade Range,District,Latitude,Longitude
0,"Marshall, Thurgood Marshall High School","45 CONKLING ST, San Francisco, CA 94124",SAN FRANCISCO,9-12,10,37.736309,-122.401649
1,San Francisco University High School,"3065 JACKSON ST, SAN FRANCISCO, CA 94115",SAN FRANCISCO,9-12,2,37.790657,-122.445351
2,Saint Ignatius College Preparatory,"2001 37TH AVE, SAN FRANCISCO, CA 94116",SAN FRANCISCO,9-12,4,37.748863,-122.495819
3,"Hearst, Phoebe Apperson Hearst Home","3045 SANTIAGO ST, SAN FRANCISCO 94116",SAN FRANCISCO,9-12,4,37.743630,-122.500053
4,"Burton, Phillip And Sala Burton High School","400 MANSELL ST, San Francisco, CA 94134",SAN FRANCISCO,9-12,9,37.721546,-122.406555
5,"Washington, George Washington High School","600 32ND AVE, San Francisco, CA 94121",SAN FRANCISCO,9-12,1,37.777905,-122.491013
6,San Francisco Waldorf High School,"470 WEST PORTAL AVE, SAN FRANCISCO, CA 94127",SAN FRANCISCO,9-12,7,37.736015,-122.471199
7,"Lincoln, Abraham Lincoln High School","2162 24TH AVE, San Francisco, CA 94116",SAN FRANCISCO,9-12,4,37.746594,-122.480240
8,Life Learning Academy Charter School,"651 8TH TI ST, SAN FRANCISCO, CA 94130",SAN FRANCISCO,9-12,6,37.825512,-122.367996
9,Gateway High School / Kipp Sf Bay Academy,"1430 SCOTT ST, San Francisco, CA 94115",SAN FRANCISCO,9-12,5,37.783264,-122.436691


In [189]:
sf_dict = {'District' : [1,2,3,4,5,6,7,8,9,10,11] , 'Population': [78060 , 67190,  75750 , 75120, 80760, 66430,  72820 , 71040, 83680, 73150,  85090 ], 'Percent Ages 5-17' : [9,7,5,11,6,5,11,6,11,15,12]}

district_data = pd.DataFrame(data = sf_dict)

district_data['Adolescent Population'] = district_data["Population"]*(district_data['Percent Ages 5-17']/100)

district_data = district_data.set_index('District')

In [126]:
sf_schools_by_district = sf_data.groupby('District').count()

sf_schools_by_district['Number of Schools'] = sf_schools_by_district['Name']

sf_schools_by_district.drop(sf_schools_by_district.columns.difference(['Number of Schools']), 1, inplace=True)

sf_schools_by_district

,Number of Schools
District,
1,1
2,6
4,3
5,8
6,2
7,5
8,3
9,3
10,2


In [216]:


address = 'San Francisco'


location = geolocator.geocode('Mount Davidson')
latitude = location.latitude
longitude = location.longitude

sf_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

district_color = {1 : 'purple', 2 : 'cyan', 3 : 'black', 4 : 'green', 5 : 'red', 6 : 'black', 7 : 'blue', 8 : 'yellow', 9 : 'orange', 10 : 'gray', 11 : 'white'}

for lat, lon, label, district in zip(sf_data['Latitude'], sf_data['Longitude'], sf_data['Name'], sf_data['District']):
    label = folium.Popup(label + ", District = " + str(district), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup= label,
        color= district_color.get(district),
        fill=True,
        fill_color= district_color.get(district),
        fill_opacity=0.7,
        parse_html=False).add_to(sf_map) 

sf_map

In [190]:
district_data['Kids Per School'] = district_data['Adolescent Population'] / sf_schools_by_district['Number of Schools']

district_data.drop([3], axis = 0)

,Population,Percent Ages 5-17,Adolescent Population,Kids Per School
District,,,,
1,78060,9,7025.4,7025.400000
2,67190,7,4703.3,783.883333
4,75120,11,8263.2,2754.400000
5,80760,6,4845.6,605.700000
6,66430,5,3321.5,1660.750000
7,72820,11,8010.2,1602.040000
8,71040,6,4262.4,1420.800000
9,83680,11,9204.8,3068.266667
10,73150,15,10972.5,5486.250000
